In [2]:
# import statements
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import glob
import os

In [3]:


# importing a sample
arr = pd.read_csv('testWave005_mijn35_R001.csv', header=None, delimiter=";")
# check the shape
input_shape = arr.shape
print(arr.iloc[0])
print(arr.iloc[1])
print(f"The input shape of your array is: {input_shape}")

0                                     Frame
1              Elapsed_Time_In_Milliseconds
2                                      Time
3                                    Hand_X
4                                    Hand_Y
                       ...                 
362       Ring_DIP_Flex_AngularAcceleration
363    Pinky_MCP_Spread_AngularAcceleration
364      Pinky_MCP_Flex_AngularAcceleration
365      Pinky_PIP_Flex_AngularAcceleration
366      Pinky_DIP_Flex_AngularAcceleration
Name: 0, Length: 367, dtype: object
0                 0
1                 0
2      00:00:00:000
3                 0
4                 0
           ...     
362        37274,14
363       -46771,95
364        27792,87
365        87709,06
366        83074,34
Name: 1, Length: 367, dtype: object
The input shape of your array is: (283, 367)


In [4]:
folder_path = 'csv_letters/*.csv'
x = []
y = []

# Example class names
class_names = ["letterA", "letterB", "letterC","letterD","letterE","letterF","letterG","letterH","letterI","letterK","letterL","letterM","letterN","letterO","letterP","letterQ","letterR","letterS","letterT","letterU","letterV","letterW","letterX","letterY"]

# Create a dictionary mapping class names to numerical labels
class_to_label = {class_name: label for label, class_name in enumerate(class_names)}
# Use glob to get all the csv files in the folder
csv_files = glob.glob(folder_path)

# Initialize an empty list to store the combined DataFrames
combined_dataframes = []
max_len = 0
for i in range(1, len(csv_files), 2):
    # Read the first file into a DataFrame
    df1 = pd.read_csv(csv_files[i], header=0, delimiter=";", usecols=list(range(0,2))+list(range(3,367)), decimal=',')
    if (len(df1)>max_len):
        max_len = len(df1)
print(f"max length is {max_len}")

max length is 315


In [5]:


# Iterate over the files two by two
for i in range(1, len(csv_files), 2):
    # Read the first file into a DataFrame
    df1 = pd.read_csv(csv_files[i], header=0, delimiter=";", usecols=list(range(0,2))+list(range(3,367)), decimal=',')
    # Get the shape of df1
    # print(f"Shape of df1: {df1.shape}")

    #get the class and add it to y
    name = os.path.splitext(csv_files[i])
    classtype = name[0].split('\\')[1][:7] #this takes letterX from the title
    numerical_label = class_to_label.get(classtype, -1)  # -1 if not found
    if numerical_label != -1:
        y.append(numerical_label)
    else:
        print("failed to add to y: " + classtype)

    # If there is a next file, read it and combine with the first
    if i+1 < len(csv_files):
        df2 = pd.read_csv(csv_files[i+1], header=0, delimiter=";", usecols=list(range(0,2))+list(range(3,367)), decimal=',')
        df2 = df2.add_prefix('right_')
    else:
        # If there is no next file, create an empty DataFrame with the same columns as df1
        df2 = pd.DataFrame(columns=df1.columns)

    # Ensure that both DataFrames have the same number of rows by filling NaN values in df2
    if len(df1) > len(df2):
        df2 = df2.reindex(df1.index, fill_value=np.nan)
    elif len(df2) > len(df1):
        df1 = df1.reindex(df2.index, fill_value=np.nan)

    # Concatenate the DataFrames
    combined_df = pd.concat([df1, df2], axis=1)
    combined_df = combined_df.reindex(range(max_len)).fillna(0)
    # Print the shape of combined_df
    print(f"Shape of combined_df: {combined_df.shape}")
    combined_dataframes.append(combined_df)

# Convert the list of DataFrames to a NumPy array
x = np.array(combined_dataframes)
x = np.asarray(x).astype('float32') # meant to fix an error when training the model
y = np.array(y)
print(x)
# print(y)



Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of c

In [14]:
model = models.Sequential()
model.add(layers.Conv1D(64, 3, activation='relu', input_shape=(315,732)))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(128, 3, activation='relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(128, 3, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(24, activation='softmax'))

In [15]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_9 (Conv1D)           (None, 313, 64)           140608    
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 156, 64)          0         
 1D)                                                             
                                                                 
 conv1d_10 (Conv1D)          (None, 154, 128)          24704     
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 77, 128)          0         
 1D)                                                             
                                                                 
 conv1d_11 (Conv1D)          (None, 75, 128)           49280     
                                                                 
 flatten_3 (Flatten)         (None, 9600)             

In [16]:
model.compile(optimizer='adam',  # we used adam during machine learning course
              loss='sparse_categorical_crossentropy',  # Multiclass classification loss
              metrics=['accuracy'])  # Track accuracy during training
print(x.size)
print(y.size)
model.fit(x,y,epochs=50)

58797900
255
Epoch 1/50
8/8 [==============================] - 2s 48ms/step - loss: 629.7651 - accuracy: 0.0392
Epoch 2/50
8/8 [==============================] - 0s 48ms/step - loss: 77.2886 - accuracy: 0.1216
Epoch 3/50
8/8 [==============================] - 1s 117ms/step - loss: 22.5225 - accuracy: 0.1843
Epoch 4/50
8/8 [==============================] - 1s 62ms/step - loss: 8.1817 - accuracy: 0.2196
Epoch 5/50
8/8 [==============================] - 0s 49ms/step - loss: 3.6199 - accuracy: 0.2431
Epoch 6/50
8/8 [==============================] - 0s 49ms/step - loss: 2.6833 - accuracy: 0.2784
Epoch 7/50
8/8 [==============================] - 0s 48ms/step - loss: 2.2239 - accuracy: 0.3412
Epoch 8/50
8/8 [==============================] - 0s 46ms/step - loss: 1.9025 - accuracy: 0.4196
Epoch 9/50
8/8 [==============================] - 0s 46ms/step - loss: 1.7671 - accuracy: 0.4471
Epoch 10/50
8/8 [==============================] - 0s 48ms/step - loss: 1.6655 - accuracy: 0.4863
Epoch 11/50